# Introduction

- Webscrape https://www.tennislive.net for pro-level match data
- Output two .csv files 
    - Single row dataframe for summary statistics dashboard
    - point by point (Pbp) level csv file for Match Viewer website => to attach timestamps using website tagger

# Install Packages

In [136]:
import requests

from bs4 import BeautifulSoup

import pandas as pd

In [137]:
page = requests.get('https://www.tennislive.net/atp/match/bernard-tomic-VS-govind-nanda/m25-tulsa-2024/')

In [138]:
soup = BeautifulSoup(page.text, 'html')

# Check Status Code
- 200 = success
- 404 = failed

In [139]:
page.status_code

200

# Single Row For Summary Statistics Dashboard

### Match Info

In [140]:
table = soup.find('table', class_ = 'table_pmatches')

date = soup.find('td', class_ = 'w50').text.strip().split(' ')[0]
round_info = soup.find_all('td', class_='w50')[1].text.strip()
player1 = soup.find_all('td', class_='w130')[0].text.strip()
player2 = soup.find_all('td', class_='w130')[1].text.strip()
score = soup.find('span', id='score').text.strip()
tournament = soup.find('td', class_='w200').find('a').text.strip()

# Create a DataFrame
data = {
    'Date': [date],
    'Round': [round_info],
    'Player 1': [player1],
    'Player 2': [player2],
    'Score': [score],
    'Tournament': [tournament]
}

df_match_info = pd.DataFrame(data)

In [141]:
df_match_info

,Date,Round,Player 1,Player 2,Score,Tournament
0,21.06.24,1/4,Bernard Tomic,Govind Nanda,"6-3, 6-1",M25 Tulsa


### Match Statistics

In [142]:
# Function to extract data for a given statistic
def extract_statistic(statistic):
    row = soup.find('td', string=statistic).parent
    player1_stat = row.find_all('td')[1].text.strip().split(' ')[0]
    player2_stat = row.find_all('td')[2].text.strip().split(' ')[0]
    return player1_stat, player2_stat

# List of statistics to extract
statistics = [
    '1st SERVE %',
    '1st SERVE POINTS WON',
    '2nd SERVE POINTS WON',
    'BREAK POINTS WON',
    'TOTAL RETURN POINTS WON',
    'TOTAL POINTS WON',
    'DOUBLE FAULTS',
    'ACES'
]

# Dictionary to hold the statistics
data = {}

# Extract and store statistics for both players
for stat in statistics:
    player1_stat, player2_stat = extract_statistic(stat)
    stat_name = stat.lower().replace(' ', '_').replace('%', 'percentage').replace('/', '_').replace('(','').replace(')','')
    data[f'{stat_name}_player1'] = player1_stat
    data[f'{stat_name}_player2'] = player2_stat

# Convert the dictionary to a DataFrame
df_stats = pd.DataFrame([data])

In [143]:
df_stats

,1st_serve_percentage_player1,1st_serve_percentage_player2,1st_serve_points_won_player1,1st_serve_points_won_player2,2nd_serve_points_won_player1,2nd_serve_points_won_player2,break_points_won_player1,break_points_won_player2,total_return_points_won_player1,total_return_points_won_player2,total_points_won_player1,total_points_won_player2,double_faults_player1,double_faults_player2,aces_player1,aces_player2
0,33/70,28/48,27/33,16/28,16/37,8/20,5/7,1/10,24/48,27/70,67/118,51/118,2,1,7,0


### Combine the DataFrames by column binding them


In [144]:
# Combine the DataFrames by column binding them
df_combined = pd.concat([df_match_info, df_stats], axis=1)

df_combined

,Date,Round,Player 1,Player 2,Score,Tournament,1st_serve_percentage_player1,1st_serve_percentage_player2,1st_serve_points_won_player1,1st_serve_points_won_player2,...,break_points_won_player1,break_points_won_player2,total_return_points_won_player1,total_return_points_won_player2,total_points_won_player1,total_points_won_player2,double_faults_player1,double_faults_player2,aces_player1,aces_player2
0,21.06.24,1/4,Bernard Tomic,Govind Nanda,"6-3, 6-1",M25 Tulsa,33/70,28/48,27/33,16/28,...,5/7,1/10,24/48,27/70,67/118,51/118,2,1,7,0


In [145]:

# Extract match information
date = soup.find('td', class_='w50').text.strip().split(' ')[0]
round_info = soup.find_all('td', class_='w50')[1].text.strip()
player1 = soup.find_all('td', class_='w130')[0].text.strip()
player2 = soup.find_all('td', class_='w130')[1].text.strip()
score = soup.find('span', id='score').text.strip()
tournament = soup.find('td', class_='w200').find('a').text.strip()

# Create a DataFrame for match info
match_data = {
    'Date': [date],
    'Round': [round_info],
    'Player 1': [player1],
    'Player 2': [player2],
    'Score': [score],
    'Tournament': [tournament]
}
df_match_info = pd.DataFrame(match_data)

# Function to extract data for a given statistic
def extract_statistic(statistic):
    row = soup.find('td', string=statistic).parent
    player1_stat = row.find_all('td')[1].text.strip().split(' ')[0]
    player2_stat = row.find_all('td')[2].text.strip().split(' ')[0]
    return player1_stat, player2_stat

# List of statistics to extract
statistics = [
    '1st SERVE %',
    '1st SERVE POINTS WON',
    '2nd SERVE POINTS WON',
    'BREAK POINTS WON',
    'TOTAL RETURN POINTS WON',
    'TOTAL POINTS WON',
    'DOUBLE FAULTS',
    'ACES'
]

# Dictionary to hold the statistics
stat_data = {}

# Extract and store statistics for both players
for stat in statistics:
    player1_stat, player2_stat = extract_statistic(stat)
    stat_name = stat.lower().replace(' ', '_').replace('%', 'percentage').replace('/', '_').replace('(','').replace(')','')
    stat_data[f'{stat_name}_player1'] = player1_stat
    stat_data[f'{stat_name}_player2'] = player2_stat

# Convert the dictionary to a DataFrame
df_stats = pd.DataFrame([stat_data])

# Combine the DataFrames by column binding them
df_combined = pd.concat([df_match_info, df_stats], axis=1)

df_combined

,Date,Round,Player 1,Player 2,Score,Tournament,1st_serve_percentage_player1,1st_serve_percentage_player2,1st_serve_points_won_player1,1st_serve_points_won_player2,...,break_points_won_player1,break_points_won_player2,total_return_points_won_player1,total_return_points_won_player2,total_points_won_player1,total_points_won_player2,double_faults_player1,double_faults_player2,aces_player1,aces_player2
0,21.06.24,1/4,Bernard Tomic,Govind Nanda,"6-3, 6-1",M25 Tulsa,33/70,28/48,27/33,16/28,...,5/7,1/10,24/48,27/70,67/118,51/118,2,1,7,0


# Multiple Links 

In [149]:
import pandas as pd

import requests

from bs4 import BeautifulSoup

In [150]:

def extract_match_data(player_name, urls):
    data_list = []
    
    for url in urls:
        # Fetch HTML content from the URL
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extract match information
        date = soup.find('td', class_='w50').text.strip().split(' ')[0]
        round_info = soup.find_all('td', class_='w50')[1].text.strip()
        player1 = soup.find_all('td', class_='w130')[0].text.strip()
        player2 = soup.find_all('td', class_='w130')[1].text.strip()
        score = soup.find('span', id='score').text.strip()
        tournament = soup.find('td', class_='w200').find('a').text.strip()
        
        # Extract statistics function
        def extract_statistic(statistic):
            row = soup.find('td', string=statistic).parent
            player1_stat = row.find_all('td')[1].text.strip().split(' ')[0]
            player2_stat = row.find_all('td')[2].text.strip().split(' ')[0]
            return player1_stat, player2_stat
        
        statistics = [
            '1st SERVE %',
            '1st SERVE POINTS WON',
            '2nd SERVE POINTS WON',
            'BREAK POINTS WON',
            'TOTAL RETURN POINTS WON',
            'TOTAL POINTS WON',
            'DOUBLE FAULTS',
            'ACES'
        ]
        
        # Check if player1 is not equal to player_name
        if player1 != player_name:
            # Swap player1 and player2
            player1, player2 = player2, player1
            
            # Reverse the score format
            score_parts = score.split(',')
            if len(score_parts) == 2:
                set1, set2 = score_parts[0].strip(), score_parts[1].strip()
                game1, game2 = set1.split('-'), set2.split('-')
                reversed_score = f'{game1[1]}-{game1[0]}, {game2[1]}-{game2[0]}'
                score = reversed_score
            if len(score_parts) == 3:
                set1, set2, set3 = score_parts[0].strip(), score_parts[1].strip(), score_parts[2].strip()
                game1, game2, game3 = set1.split('-'), set2.split('-'), set3.split('-')
                reversed_score = f'{game1[1]}-{game1[0]}, {game2[1]}-{game2[0]}, {game3[1]}-{game3[0]}'
                score = reversed_score
                
            
            # Extract statistics for swapped players
            stat_data = {}
            for stat in statistics:
                player2_stat, player1_stat = extract_statistic(stat)
                stat_name = stat.lower().replace(' ', '_').replace('%', 'percentage').replace('/', '_').replace('(','').replace(')','')
                stat_data[f'{stat_name}_player1'] = player1_stat
                stat_data[f'{stat_name}_player2'] = player2_stat
        else:
            # Extract statistics for original players
            stat_data = {}
            for stat in statistics:
                player1_stat, player2_stat = extract_statistic(stat)
                stat_name = stat.lower().replace(' ', '_').replace('%', 'percentage').replace('/', '_').replace('(','').replace(')','')
                stat_data[f'{stat_name}_player1'] = player1_stat
                stat_data[f'{stat_name}_player2'] = player2_stat
        
        # Append match data to the list
        match_data = {
            'Date': date,
            'Round': round_info,
            'Player 1': player1,
            'Player 2': player2,
            'Score': score,
            'Tournament': tournament,
            **stat_data
        }
        data_list.append(match_data)
    
    # Convert list of dictionaries to DataFrame
    df = pd.DataFrame(data_list)
    
    # Sort DataFrame by Date in descending order
    df['Date'] = pd.to_datetime(df['Date'], format='%d.%m.%y')
    
    df = df.sort_values(by='Date', ascending=True).reset_index(drop=True)
    
    return df


### INPUT (playerName) and (links) HERE

In [135]:
# Example usage:
player_name = "Rudy Quan"

urls = [
    'https://www.tennislive.net/atp/match/rudy-quan-VS-shintaro-imai/little-rock-challenger-2024/',
    'https://www.tennislive.net/atp/match/rudy-quan-VS-filip-peliwo/little-rock-challenger-2024/',
    'https://www.tennislive.net/atp/match/rudy-quan-VS-stefan-kozlov/little-rock-challenger-2024/',
    'https://www.tennislive.net/atp/match/yuta-shimizu-VS-rudy-quan/little-rock-challenger-2024/',
    'https://www.tennislive.net/atp/match/rudy-quan-VS-andres-andrade/little-rock-challenger-2024/'
]

df = extract_match_data(player_name, urls)
df


,Date,Round,Player 1,Player 2,Score,Tournament,1st_serve_percentage_player1,1st_serve_percentage_player2,1st_serve_points_won_player1,1st_serve_points_won_player2,...,break_points_won_player1,break_points_won_player2,total_return_points_won_player1,total_return_points_won_player2,total_points_won_player1,total_points_won_player2,double_faults_player1,double_faults_player2,aces_player1,aces_player2
0,2024-05-26,q 1,Rudy Quan,Shintaro Imai,"4-6, 6-4, 6-2",Little Rock,55/76,54/118,36/55,34/54,...,6/19,4/6,55/118,30/76,101/194,93/194,2,9,0,15
1,2024-05-27,qual.,Rudy Quan,Filip Peliwo,"6-4, 6-2",Little Rock,36/51,23/49,20/36,12/23,...,7/7,4/8,32/49,25/51,58/100,42/100,1,4,0,1
2,2024-05-28,1st round,Rudy Quan,Stefan Kozlov,"6-1, 7-5",Little Rock,42/62,41/58,25/42,19/41,...,6/11,3/6,34/58,27/62,69/120,51/120,1,3,0,0
3,2024-05-30,2nd round,Rudy Quan,Andres Andrade,"6-3, 3-6, 6-3",Little Rock,59/84,44/89,32/59,25/44,...,8/17,6/9,48/89,42/84,90/173,83/173,7,10,1,3
4,2024-05-31,1/4,Rudy Quan,Yuta Shimizu,"4-6, 3-6",Little Rock,64/80,37/55,29/64,21/37,...,4/7,7/16,25/55,44/80,61/135,74/135,2,1,0,4


In [117]:
    
# score = "6-4, 6-2"
    
# # Reverse the score format
# score_parts = score.split(',')
# if len(score_parts) == 2:
#     set1, set2 = score_parts[0].strip(), score_parts[1].strip()
#     game1, game2 = set1.split('-'), set2.split('-')
#     reversed_score = f'{game1[1]}-{game1[0]}, {game2[1]}-{game2[0]}'
#     score = reversed_score
        
# scoredd

'4-6, 2-6'

# Point by Point Dataframe

In [151]:
page = requests.get('https://www.tennislive.net/atp/match/yuta-shimizu-VS-rudy-quan/little-rock-challenger-2024/')

In [152]:
soup = BeautifulSoup(page.text, 'html')

# Check Status Code
- 200 = success
- 404 = failed

In [153]:
page.status_code

200

In [154]:
table = soup.find_all('table', class_ = 'table_stats_match')


In [155]:
table[1]

<table class="table_stats_match"><tr class="mp_tour_head"><td colspan="3" width="100%">1</td></tr><tr><td class="mp_serve" width="40%"></td><td class="mp_info_txt" width="18%">0-0</td><td class="mp_serve" width="40%">Rudy Quan <img alt="Rudy Quan serve" height="8" src="https://www.tennislive.net/styles/images/tennis_ball.gif" title="Rudy Quan serve" width="8"/></td></tr><tr><td class="mp_15" colspan="3" width="99%">0-0, 0-15, 15-15, 15-30, 30-30, 30-40, 40-40, A-40<span title="Break point">[BP]</span>, 40-40, 40-A, 40-40, 40-A, 40-40, 40-A, 40-40, 40-A, 40-40, 40-A, 40-40, 40-A</td></tr><tr><td class="mp_serve" width="40%">Yuta Shimizu <img alt="Yuta Shimizu serve" height="8" src="https://www.tennislive.net/styles/images/tennis_ball.gif" title="Yuta Shimizu serve" width="8"/></td><td class="mp_info_txt" width="18%">0-1</td><td class="mp_serve" width="40%"></td></tr><tr><td class="mp_15" colspan="3" width="99%">0-0, 0-15, 0-30, 15-30, 15-40<span title="Break point">[BP]</span></td></tr>

In [156]:
# Initialize lists to store the data
point_scores = []
server_names = []
is_break_points = []

# Find all rows with point scores
point_rows = soup.find_all('tr', class_='mp_15')
serve_rows = soup.find_all('tr')

# Initialize current server name
current_server = None

# Iterate over each serve row to track the server changes
for row in serve_rows:
    # Check if the row is a serve row with a server name
    serve_cell = row.find('td', class_='mp_serve')
    if serve_cell and serve_cell.text.strip():
        current_server = serve_cell.text.strip().split(' ')[0]

    # Check if the row contains points
    point_cell = row.find('td', class_='mp_15')
    if point_cell:
        point_score_text = point_cell.text.strip()
        point_score_list = point_score_text.split(', ')
        
        for point in point_score_list:
            point_scores.append(point)
            server_names.append(current_server)
            
            # Determine break point status
            if '[BP]' in point:
                is_break_point = True
                point = point.replace('[BP]', '').strip()
            else:
                is_break_point = False
            is_break_points.append(is_break_point)

# Create the DataFrame
df = pd.DataFrame({
    'pointScore': point_scores,
    'serverName': server_names,
    'isBreakPoint': is_break_points
})

df

# df.to_csv('tennis_match_points.csv', index=False)


,pointScore,serverName,isBreakPoint
0,0-0,None,False
1,0-15,None,False
2,15-15,None,False
3,15-30,None,False
4,30-30,None,False
...,...,...,...
128,15-0,Yuta,False
129,30-0,Yuta,False
130,40-0[BP],Yuta,True
131,40-15[BP],Yuta,True


In [84]:
# Initialize lists to store the data
point_scores = []
server_names = []
is_break_points = []
game_scores = []

# Initialize current server name and game score
current_server = None
current_game_score = '0-0'
game_score_counter = 0

# Find all rows with point scores
point_rows = soup.find_all('tr', class_='mp_15')
serve_rows = soup.find_all('tr')

# Iterate over each serve row to track the server changes
for row in serve_rows:
    # Check if the row is a serve row with a server name
    serve_cell = row.find('td', class_='mp_serve')
    if serve_cell and serve_cell.text.strip():
        current_server = serve_cell.text.strip().split(' ')[0]

    # Check if the row contains points
    point_cell = row.find('td', class_='mp_15')
    if point_cell:
        point_score_text = point_cell.text.strip()
        point_score_list = point_score_text.split(', ')
        
        for point in point_score_list:
            # Update game score if the point score is 0-0
            if point == '0-0':
                current_game_score = f'{game_score_counter}-0'
                game_score_counter += 1

            point_scores.append(point)
            server_names.append(current_server)
            game_scores.append(current_game_score)
            
            # Determine break point status
            if '[BP]' in point:
                is_break_point = True
                point = point.replace('[BP]', '').strip()
            else:
                is_break_point = False
            is_break_points.append(is_break_point)

# Create the DataFrame
df = pd.DataFrame({
    'pointScore': point_scores,
    'serverName': server_names,
    'isBreakPoint': is_break_points,
    'gameScore': game_scores
})

print(df)

# Optionally, save the DataFrame to a CSV file
df.to_csv('tennis_match_points.csv', index=False)


    pointScore serverName  isBreakPoint gameScore
0          0-0       None         False       0-0
1         15-0       None         False       0-0
2         30-0       None         False       0-0
3     40-0[BP]       None          True       0-0
4    40-15[BP]       None          True       0-0
..         ...        ...           ...       ...
113      30-15    Bernard         False      15-0
114      40-15    Bernard         False      15-0
115      40-30    Bernard         False      15-0
116      40-40    Bernard         False      15-0
117       A-40    Bernard         False      15-0

[118 rows x 4 columns]


# Example Dataframe

In [147]:
import pandas as pd

# Define the column names
columns = [
    "Date", "Match", "Duration", "Player Name", "Total serves", "Aces", "1st Serve In %",
    "2nd Serve In %", "1st Serve Won %", "2nd Serve Won %", "Double Faults", "1st serve Ad",
    "1st serve De", "1st serve Ad %", "1st serve De %", "2nd serve Ad %", "2nd serve De %",
    "Double Fault Ad %", "Double Fault De %", "Average Rally Count", "3 Shot Rally Count",
    "Break Points", "Break Points Won", "Break Points Won %", "Total Points Won on Serve",
    "Break Points Saved %", "Total Groundstrokes", "Groundstrokes Won", "Total Returns",
    "Total Returns Won", "Volley Count", "Volley Winner Count", "At Net Count", "Total Slices",
    "Number of Dropshots", "Forehand/Backhand Errors (Count)"
]

# Create an empty DataFrame with one row
df = pd.DataFrame(columns=columns, index=[0])



In [148]:
list(df)

['Date',
 'Match',
 'Duration',
 'Player Name',
 'Total serves',
 'Aces',
 '1st Serve In %',
 '2nd Serve In %',
 '1st Serve Won %',
 '2nd Serve Won %',
 'Double Faults',
 '1st serve Ad',
 '1st serve De',
 '1st serve Ad %',
 '1st serve De %',
 '2nd serve Ad %',
 '2nd serve De %',
 'Double Fault Ad %',
 'Double Fault De %',
 'Average Rally Count',
 '3 Shot Rally Count',
 'Break Points',
 'Break Points Won',
 'Break Points Won %',
 'Total Points Won on Serve',
 'Break Points Saved %',
 'Total Groundstrokes',
 'Groundstrokes Won',
 'Total Returns',
 'Total Returns Won',
 'Volley Count',
 'Volley Winner Count',
 'At Net Count',
 'Total Slices',
 'Number of Dropshots',
 'Forehand/Backhand Errors (Count)']